## Driver

Drivers are virtualized devices through which applications can interact and initiate I/O operations on the target device.
In GPOS, drivers are part of kernel space and applications are part of user space

A Driver executes in the context of the application.

## Driver Code

It is divided into two parts.

#### Kernel Specific code or User Interaction Code: 
Application interfaces with this code to initiate any operation.
Therefore Windows, Linux will have different kernel specific code.
Writing kernel specific involves kernel programming, kernel testing.

#### Hardware Specific code: 
This code performs the operation on the target device.
This code will be completely hardware specific like device/bus specific.

## User Ineraction Code can be written in three ways:

These are the three different ways of allowing the application to interface with the driver.
Character Driver Approach
Block Driver Approach
Network Driver Approach
